STEP 1 — Load & Preview Data

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler


In [18]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

columns = [
    "age", "workclass", "fnlwgt", "education", "education_num",
    "marital_status", "occupation", "relationship", "race", "sex",
    "capital_gain", "capital_loss", "hours_per_week",
    "native_country", "income"
]

df = pd.read_csv(url, names=columns, sep=", ", engine="python")
df.head()


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


STEP 2 — Basic Data Inspection

In [19]:
df.shape
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


STEP 3 — Missing Value Handling

3.1 Check Missing Values

(In this dataset, missing values are represented by "?")

In [20]:
(df == '?').sum()


,0
age,0
workclass,1836
fnlwgt,0
education,0
education_num,0
marital_status,0
occupation,1843
relationship,0
race,0
sex,0


3.2 Replace "?" with NaN

In [21]:
df.replace('?', np.nan, inplace=True)



3.3 Fill workclass with Mode

In [22]:
df['workclass'] = df['workclass'].fillna(df['workclass'].mode()[0])


3.4 Fill occupation with Mode

In [23]:
df['occupation'] = df['occupation'].fillna(df['occupation'].mode()[0])


3.5 Fill native_country with Mode

In [24]:
df['native_country'] = df['native_country'].fillna(df['native_country'].mode()[0])


3.6 Final Missing Value Check

In [25]:
df.isnull().sum()


,0
age,0
workclass,0
fnlwgt,0
education,0
education_num,0
marital_status,0
occupation,0
relationship,0
race,0
sex,0


STEP 4 — FEATURE ENGINEERING (3 FEATURES)

FE 1 — Capital Difference

In [26]:
df['capital_diff'] = df['capital_gain'] - df['capital_loss']
df['capital_diff']


,capital_diff
0,2174
1,0
2,0
3,0
4,0
...,...
32556,0
32557,0
32558,0
32559,0


FE 2 — Working Intensity

In [27]:
df['work_intensity'] = df['hours_per_week'] / df['age']
df['work_intensity']


,work_intensity
0,1.025641
1,0.260000
2,1.052632
3,0.754717
4,1.428571
...,...
32556,1.407407
32557,1.000000
32558,0.689655
32559,0.909091


FE 3 — Education Category

In [28]:
df['education_level'] = df['education_num'].apply(
    lambda x: 'Low' if x <= 8 else 'Medium' if x <= 12 else 'High'
)
df['education_level']


,education_level
0,High
1,High
2,Medium
3,Low
4,High
...,...
32556,Medium
32557,Medium
32558,Medium
32559,Medium


STEP 5 — ENCODING (Categorical → Numeric)

5.1 One-Hot Encoding

In [29]:
categorical_cols = [
    'workclass', 'education', 'marital_status',
    'occupation', 'relationship', 'race',
    'sex', 'native_country', 'education_level'
]

df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)


5.2 Encode Target Variable

In [30]:
df_encoded['income'] = df_encoded['income'].map({'<=50K': 0, '>50K': 1})


In [31]:
df.shape, df_encoded.shape


((32561, 18), (32561, 102))

STEP 6 — FEATURE SCALING

In [32]:
scale_cols = [
    'age', 'fnlwgt', 'education_num',
    'capital_gain', 'capital_loss',
    'hours_per_week', 'capital_diff', 'work_intensity'
]

scaler = StandardScaler()
df_encoded[scale_cols] = scaler.fit_transform(df_encoded[scale_cols])


In [33]:
df_encoded[scale_cols].describe()


,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,capital_diff,work_intensity
count,3.256100e+04,3.256100e+04,3.256100e+04,3.256100e+04,3.256100e+04,3.256100e+04,3.256100e+04,3.256100e+04
mean,-2.705915e-17,-1.001625e-16,1.471887e-16,1.309314e-17,1.016900e-16,-1.549355e-17,1.745752e-17,-3.587520e-16
std,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00
min,-1.582206e+00,-1.681631e+00,-3.529656e+00,-1.459205e-01,-2.166595e-01,-3.194030e+00,-7.216138e-01,-2.362655e+00
25%,-7.757679e-01,-6.816910e-01,-4.200596e-01,-1.459205e-01,-2.166595e-01,-3.542945e-02,-1.336701e-01,-6.792775e-01
50%,-1.159546e-01,-1.082193e-01,-3.136003e-02,-1.459205e-01,-2.166595e-01,-3.542945e-02,-1.336701e-01,-1.097137e-01
75%,6.904838e-01,4.478765e-01,7.460392e-01,-1.459205e-01,-2.166595e-01,3.695194e-01,-1.336701e-01,6.022411e-01
max,3.769612e+00,1.226856e+01,2.300838e+00,1.339458e+01,1.059351e+01,4.742967e+00,1.336352e+01,8.295849e+00


STEP 7 — Save Final Dataset

In [34]:
df_encoded.to_csv("adult_income_preprocessed.csv", index=False)
